In [47]:
import torch 
print (torch.cuda.is_available())


False


Installing Packages

In [52]:
#!pip install -r /content/requirements.txt
!pip install -r requirements.txt

  Using cached torch-1.8.1-cp36-cp36m-win_amd64.whl (190.5 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.10.1
    Uninstalling torch-1.10.1:
      Successfully uninstalled torch-1.10.1


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Отказано в доступе: 'c:\\python\\python36\\lib\\site-packages\\~-rch\\lib\\asmjit.dll'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\python\python36\python.exe -m pip install --upgrade pip' command.


In [48]:
import pandas as pd

In [49]:
train_data = pd.read_csv('train.csv')
valid_data = pd.read_csv('valid.csv')
test_data  = pd.read_csv('test.csv')
#train_data = pd.read_csv('db10-1.csv') # broader + not broader

In [53]:
from bert_dataset import CustomDataset
from bert_classifier import BertClassifier

In [55]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [56]:
from bert_classifier import BertClassifier
model = BertClassifier(data_processor, 
                       num_labels, 
                       bert_config_file,
                       max_seq_length, 
                       vocab_file, 
                       tensorboard_dir, 
                       init_checkpoint, 
                       keep_checkpoint_max, 
                       use_GPU, 
                       label_smoothing, 
                       cycle)
#model.predict(file_path='./data/test.tsv', predict_batch_size=128, output_dir='./predict')
# Or single sample inference
# prob = model.predict(input_example=input_example)

NameError: name 'data_processor' is not defined

Initialize BERT classifier

In [63]:
classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        n_classes=2,
        epochs=2,
        model_save_path='bert.pt',
        
)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

Prepare data and helpers for train and evlauation

In [64]:
classifier.preparation(
        X_train=list(train_data['text']),
        y_train=list(train_data['label']),
        X_valid=list(valid_data['text']),
        y_valid=list(valid_data['label'])
    )

Train loop

In [65]:
classifier.train()

Epoch 1/2
Train loss 0.42354585234160597 accuracy 0.8873295910184442
Val loss 0.4121066030478353 accuracy 0.8997912317327766
----------
Epoch 2/2
Train loss 0.30102264597629586 accuracy 0.9296311146752205
Val loss 0.37973104637844823 accuracy 0.918580375782881
----------


Check test data

In [66]:
texts = list(test_data['text'])
labels = list(test_data['label'])

predictions = [classifier.predict(t) for t in texts]

In [67]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.933658620099298, recall: 0.9333239560840607, f1score: 0.9327454074425223
